# EasyVVUQ - Vector Quantities of Interest

**Author**: Vytautas Jancauskas, LRZ (jancauskas@lrz.de)

It is often the case that simulation outputs are vector valued and represent changes over time in whatever phenomenon that is simulated. Here we examine how to analyse these cases with EasyVVUQ. As an example we use the following model from epidemiology - [SIR](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology#The_SIR_model). It compartmentalises the population into the following groups: ($S$)usceptible, ($I$)nfected and ($R$)emoved. There are four input parameters to out model. They are initial number of susceptible people $S_0$, initial number of infected people $I_0$, transmission rate $\beta$ and recovery rate $\gamma$. The system is governed by three differential equations below. The number of suspectible people is reduced by them getting infected (and thus moving to $I$) at rate specified by $\beta$. The number of infected people increases similarly. Finally the number of recovered people increases depending on the recovery rate and the number of people currently infected.

$${\frac  {dS}{dt}}=-{\frac  {\beta SI}{N}}$$

$${\displaystyle {\frac {dI}{dt}}={\frac {\beta SI}{N}}-\gamma I}$$

$${\displaystyle {\frac {dR}{dt}}=\gamma I}$$

The simulation is run as shown below, where ```input_file``` is a JSON file of the following format: ```{"outfile": "$outfile", "S0": $S0, "I0": $I0, "beta": $beta, "gamma": $gamma, "iterations": $iterations}```

```sir <input_file>```

Lets try and run it.

In [1]:
!echo '{"outfile": "output.csv", "S0": 997, "I0": 3, "beta": 0.2, "gamma": 0.04, "iterations": 100}' > input.json

In [2]:
!./sir input.json

The simulation code outputs a CSV file with a specified name. We can try opening it up and printing the first 10 lines.

In [3]:
!head output.csv

S,I,R,r0,t
996.4018,3.47784108,0.1391136432,4982.009,1
995.7087475755611,4.0312978050411745,0.300365555401647,4978.543737877805,2
994.905980318425,4.672165937353743,0.4872521928957967,4974.5299015921255,3
993.9763679472093,5.4140230672294996,0.7038131155849767,4969.8818397360465,4
992.9001871310173,6.27247777694467,0.9547122266627635,4964.500935655087,5
991.654756898651,7.2654467105291545,1.2453300950839297,4958.273784493255,6
990.2140324288114,8.413460157063296,1.5818685013664615,4951.070162144058,7
988.548155936764,9.739995672478827,1.9714683282656145,4942.740779683821,8
986.6229648025937,11.27183768247915,2.4223418355647803,4933.11482401297,9


Finally lets plot the results.

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('output.csv')

As we can see it shows the number of infected people increasing at first and then decreasing as the number of people who have recovered rises. Number of susceptible and recovered people also changes in a way that makes intuitive sense.

In [5]:
plt.plot(df['S'], label='S')
plt.plot(df['I'], label='I')
plt.plot(df['R'], label='R')
plt.legend()

So let us see what EasyVVUQ can tell us about the distribution of the number of infected people over time. We will not go into the details of the EasyVVUQ API this time. Only where it differs from the [basic concepts](./basic_tutorial.ipynb) tutorial.

In [6]:
import easyvvuq as uq
import chaospy as cp
import os
import matplotlib.pyplot as plt

In [7]:
params = {
    "S0": {"type": "float", "default": 997}, 
    "I0": {"type": "float", "default": 3}, 
    "beta": {"type": "float", "default": 0.2}, 
    "gamma": {"type": "float", "default": 0.04, "min": 0.0, "max": 1.0},
    "iterations": {"type": "integer", "default": 100},
    "outfile": {"type": "string", "default": "output.csv"}
}

Encoder is the same as in the basic concepts tutorial, essentially. However, since the simulation outputs a CSV file with the evolution of output variables over time we want the ```SimpleCSV``` decoder. The arguments to it should be self explanatory.

In [8]:
encoder = uq.encoders.GenericEncoder(template_fname='sir.template', delimiter='$', target_filename='input.json')
decoder = uq.decoders.SimpleCSV(target_filename='output.csv', output_columns=['I'])
actions = uq.actions.local_execute(encoder, os.path.abspath('sir') + ' input.json', decoder)

In [9]:
campaign = uq.Campaign(name='sir', params=params, actions=actions)

We assume that the infection rate $\beta$ is uniformly distributed between 0.15 and 0.25 and the recovery rate $\gamma$ is normally distributed with mean 0.04 and small variance.

In [10]:
vary = {
    "beta": cp.Uniform(0.15, 0.25),
    "gamma": cp.Normal(0.04, 0.01),
}

For this tutorial we will use Polynomial Chaos Expansion method. However, both [SCSampler](https://easyvvuq.readthedocs.io/en/dev/easyvvuq.sampling.html#module-easyvvuq.sampling.stochastic_collocation) and [QMCSampler](https://easyvvuq.readthedocs.io/en/dev/easyvvuq.sampling.html#module-easyvvuq.sampling.qmc) would work as well and might be preferable depending on the case.

In [11]:
campaign.set_sampler(uq.sampling.PCESampler(vary=vary, polynomial_order=5))

In [12]:
execution = campaign.execute()

In [13]:
execution.progress()

{'ready': 28, 'active': 8, 'finished': 0, 'failed': 0}

In [14]:
execution.collate(progress_bar=True)
result = campaign.analyse(qoi_cols=['I'])

100%|██████████| 36/36 [00:07<00:00,  4.72it/s]


We can now see the results of the analysis. One thing to try would be to plot the first order sobol indices over time. This shows us how much influence the two parameters (```beta``` and ```gamma```) have over the number of people infected over time ```t```. The ```higher orders``` line is meant to represent the influence of the interactions between the input variables. However it is negligible in this case.

In [15]:
result.plot_sobols_first('I', xlabel='t')

<AxesSubplot:xlabel='t', ylabel='First Order Sobol Index'>

Finally we can try and visualize certain aspects of the distribution for our quantity of interest (number of infected people) over time. In this plot we will plot the mean value, standard deviation and 0.01 and 0.99 quantiles. This is with respect to the input variable distributions we have specified in the ```vary``` dictionary.

In [16]:
result.plot_moments('I', xlabel='t')

<AxesSubplot:xlabel='t', ylabel='I'>

If you want to access all of this data for processing you can use the following methods. Due to large output I have not evaluated these cells.

In [17]:
result.sobols_first('I')

{'beta': array([0.89204991, 0.89185266, 0.89149001, 0.89096097, 0.89026466,
        0.88940042, 0.88836788, 0.88716713, 0.88579887, 0.88426465,
        0.88256715, 0.88071051, 0.87870074, 0.87654612, 0.87425765,
        0.87184944, 0.86933898, 0.86674715, 0.86409788, 0.8614171 ,
        0.85873111, 0.85606382, 0.85343316, 0.85084649, 0.84829549,
        0.84575091, 0.84315795, 0.84043276, 0.83746062, 0.83409584,
        0.83016322, 0.82546019, 0.81975914, 0.81280885, 0.80433458,
        0.7940368 , 0.78158862, 0.76663314, 0.74878204, 0.72761773,
        0.7027023 , 0.67359755, 0.63990145, 0.60130728, 0.55769073,
        0.50922621, 0.45652411, 0.40076392, 0.34377714, 0.28801728,
        0.23636068, 0.19172748, 0.15659244, 0.13253095, 0.11995903,
        0.11815331, 0.12551813, 0.13997798, 0.15935905, 0.18167041,
        0.20525921, 0.22885867, 0.25156611, 0.272786  , 0.29216284,
        0.30951812, 0.32479749, 0.33802971, 0.34929615, 0.35870914,
        0.366397  , 0.37249406, 0.377134

In [18]:
result.describe('I', 'mean')

array([  3.4778336 ,   4.03404206,   4.68171534,   5.43612205,
         6.31505352,   7.3392123 ,   8.5326462 ,   9.92322633,
        11.54316403,  13.42955585,  15.62493745,  18.17781631,
        21.14313804,  24.58262204,  28.56487862,  33.16519322,
        38.46483541,  44.54972656,  51.50828773,  59.42829956,
        68.39265292,  78.47396641,  89.72820311, 102.18763148,
       115.85371859, 130.69076976, 146.62126765, 163.52384337,
       181.23458606, 199.55197143, 218.24513987, 237.06470706,
       255.75488712, 274.06555177, 291.76296652, 308.63828094,
       324.51329838, 339.24349061, 352.71856713, 364.86111453,
       375.62389166, 384.98633302, 392.95071792, 399.53834415,
       404.78592878, 408.74236162, 411.46586385, 413.02155449,
       413.47939812, 412.91249216, 411.39564756, 409.00421782,
       405.81313627, 401.89612761, 397.32506563, 392.16945484,
       386.49601832, 380.36837781, 373.84681454, 366.98810159,
       359.84539945, 352.46820727, 344.90236319, 337.19

In [19]:
result.describe('I', 'std')

array([9.13081930e-02, 2.11560354e-01, 3.67777061e-01, 5.68492124e-01,
       8.24044420e-01, 1.14691458e+00, 1.55210823e+00, 2.05758473e+00,
       2.68472560e+00, 3.45882970e+00, 4.40961189e+00, 5.57166667e+00,
       6.98483857e+00, 8.69441463e+00, 1.07510228e+01, 1.32100846e+01,
       1.61306351e+01, 1.95733000e+01, 2.35972171e+01, 2.82557340e+01,
       3.35908224e+01, 3.96263450e+01, 4.63605930e+01, 5.37588577e+01,
       6.17471385e+01, 7.02083075e+01, 7.89820280e+01, 8.78693628e+01,
       9.66423075e+01, 1.05057574e+02, 1.12873064e+02, 1.19864881e+02,
       1.25842595e+02, 1.30660906e+02, 1.34226572e+02, 1.36500409e+02,
       1.37494923e+02, 1.37268669e+02, 1.35918592e+02, 1.33571532e+02,
       1.30375825e+02, 1.26493586e+02, 1.22094009e+02, 1.17347705e+02,
       1.12421962e+02, 1.07476667e+02, 1.02660572e+02, 9.81076004e+01,
       9.39330242e+01, 9.02295425e+01, 8.70636263e+01, 8.44728002e+01,
       8.24646905e+01, 8.10185006e+01, 8.00890700e+01, 7.96129967e+01,
      

In [20]:
result.describe('I', '1%')

array([  3.29852547,   3.62647216,   3.98668024,   4.38225166,
         4.81657168,   5.29333157,   5.81655257,   6.39061079,
         7.02026301,   7.71067311,   8.46743864,   9.29661703,
        10.20475056,  11.19888908,  12.28660919,  13.47602849,
        14.77581412,  16.1951858 ,  17.74391638,  19.43233693,
        21.27135854,  23.27281793,  25.45579714,  27.81784671,
        30.38158209,  33.16280847,  36.17864098,  39.44701466,
        42.98560529,  46.81014475,  50.91298283,  55.33142179,
        60.05388907,  65.0750717 ,  70.40555395,  75.96874042,
        81.88559425,  88.17559439,  94.76253446, 101.7906746 ,
       109.13806097, 116.78335595, 124.53367842, 132.57163759,
       140.93801848, 149.3901359 , 156.94171257, 165.42669799,
       173.68721157, 182.3618398 , 190.73479682, 198.98726743,
       206.78659683, 214.04149374, 220.33043481, 226.35521018,
       228.79596829, 226.72347854, 221.53235772, 215.16015523,
       207.36520066, 198.98675867, 190.26158561, 181.44

In [21]:
result.describe('I', '99%')

array([  3.65473544,   4.45159463,   5.42105695,   6.59995928,
         8.03273578,   9.77237069,  11.88245812,  14.43937619,
        17.53357297,  21.2718391 ,  25.77933367,  31.20140399,
        37.70491606,  45.47868804,  54.73247024,  65.69375369,
        78.60154364,  93.73168347, 111.27448377, 131.6344494 ,
       154.7857425 , 180.86403334, 209.58681046, 241.43954559,
       274.88582043, 309.82691674, 345.72971088, 382.13983461,
       418.49281379, 453.56094937, 489.24298587, 517.06485735,
       547.05623467, 568.58269584, 589.97745129, 605.24056993,
       615.80919014, 624.13853541, 632.23797939, 637.78993505,
       643.17971248, 646.1495388 , 647.29732844, 647.22908542,
       644.68533053, 640.67413635, 640.04781522, 638.15402871,
       636.31941958, 633.30742371, 629.6295947 , 623.59809917,
       617.39323537, 612.19873087, 610.48310439, 606.38344966,
       602.70792655, 596.66956347, 591.12890481, 586.73621   ,
       582.17974134, 575.14420168, 568.66783872, 562.49